<a href="https://colab.research.google.com/github/Geonho-Park/-/blob/main/20201114_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 국민청원 데이터 가져오기


- 참고:
  - [국민청원 데이터셋 CSV](https://newhiwoong.github.io/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B)
  - [청와대 국민청원 https://www1.president.go.kr/petitions](https://www1.president.go.kr/petitions)
  - [Using Selenium with Google Colaboratory https://darektidwell.com/using-selenium-with-google-colaboratory/](https://darektidwell.com/using-selenium-with-google-colaboratory/)
  - [https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path](https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path)
 
- 참고링크 중 "국민청원 데이터셋 CSV"의 내용을 토대로 수정하여 작성
- 1건당 1-2초(google colab에서 더 느림, 실 사용은 로컬을 권장함)

## 확인된 에러의 경우 
1. 없는 페이지
2. 관리자 차단 페이지

In [1]:
import sys

# install chromium, its driver, and selenium
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import re

if 'ipykernel' in sys.modules:
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [3]:
def get_petition(wd, number):
    base_url='https://www1.president.go.kr/petitions/'
    try:
        wd.get(base_url+str(number))
        element = WebDriverWait(wd, 2).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'petitionsView_title')))
        #bs = BeautifulSoup(wd.page_source, 'html.parser')
        bs = BeautifulSoup(wd.page_source, 'lxml')

        progress = bs.find('div',{'class':'petitionsView_progress'}).get_text().strip()
        title = bs.find('h3',{'class':'petitionsView_title'}).get_text().strip()
        like = bs.find('h2',{'class':'petitionsView_count'}).get_text().strip()
        like = int(''.join(re.findall('\d+', str(like))))
        if progress == '답변완료':
            contents = bs.find_all('div',{'class':'View_write'})[1].get_text().strip()
            answer = bs.find_all('div',{'class':'pr_tk25'})[1].get_text().strip()[4:].strip()
        else:
            contents = bs.find('div',{'class':'View_write'}).get_text().strip()
            answer = ''
        info = bs.find('ul',{'class':'petitionsView_info_list'}).get_text().split('\n')
        category = info[1][4:]
        sday = info[2][4:]
        eday = info[3][4:]
        proponent = info[4][3:].split()[0]
        return {'number':number, 'progress':progress, 'title':title,  'like':like,
                'contents':contents, 'answer':answer, 'category':category,
                'sday':sday, 'eday':eday, 'proponent':proponent}
    except Exception as e:
        print(number, e)
        return {'number':number, 'progress':'error', 'title':'',  'like':0,
                'contents':str(e), 'answer':'', 'category':'',
                'sday':'', 'eday':'', 'proponent':''}

In [4]:
if __name__ == '__main__' or 'google.colab' in sys.modules:
    wd = webdriver.Chrome('chromedriver',options=options)
    print(get_petition(wd, 300))
    print(get_petition(wd, 582349))
    wd.quit()

{'number': 300, 'progress': '청원종료', 'title': '대기업의 잘못된 인터넷 개인가입자 민원 처리방식과 위약금', 'like': 0, 'contents': '저는 SK인터넷,TV를 가입해서 사용하고 있는 개인 가입자 입니다.8/22일 휴가를 마치고 집으로 오니 인터넷,TV가 안되어서 SK 장애부서인 106번 으로전화하여 수리요청을 했습니다.상담원이 관할지역 기사님을 알아보고는 오늘은 기사님들이 스케줄이 안되어서 장애처리를 내일 해야 된다고 하였습니다.그래서 오늘 아이들 인터넷으로 숙제도해야되고 TV시청도 해야되니 오늘 늦더라도 고쳐달라고 말하니,조금뒤 기술부서 실장님이 전화를 주셔서 어떻게 해도 금일중으로는  기사님을 수배할수 없기 때문에 장애처리를 할수 없고 내일 오전10시즘에 처리 할수 있다고 말했습니다.A/S기사님들은 오전에 그날 A/S처리  스케줄을 다잡고 나가는데 그러면 소비자 들은 장애가 나면 무조건 다음날 A/S처리를 봤아야 되니까? 소비자 잘못이 아니라 SK장비때문에 장애가 났는데 이거는 SK에서 긴급 장애처리 기사를 더고용해서 당일 장애건도 그날A/S처리를 해줘야 되는거 아닙니까.그리고 SK잘못때문에 장애가 났는데 당일 처리를 못해주면 인터넷 해지를 요청하니 수십만원의 위약금을 물어야 된다고 하는데 이것은 잘못된 처방 아닙니까.자기들 장비때문에 장애가나서 사용못하고 그래서 수리요청하니 당일 처리가 힘들고,그러면 해지하려하니 위약금 내라고 하고,완전히 개인가입자는 봉입니까.그리고 처음 인터넷가입시 장애처리 문제에 대해서는 설명도 없었는데 장애가 나고나니 스케줄 잡아서 내일 A/S받든지 아님 나갈 사람이 없어방법이 없다.알아서 해라라는 식으로 답변을 듣기만 해야되나요.장애신고 하는 사람들은 지금 인터넷,TV가 안되어서 전화를 하는 사람들이지 미리 고장날것을 알고 전화해서 예약스케줄을 잡는 사람들이 아닙니다.SK인터넷 장애처리부서는 생각좀 하시고 일합시다.', 'answer': '', 'category': '기타', 's

## 사용예

In [5]:
import pandas as pd

## 청와대 국민청원: https://www1.president.go.kr/petitions

def get_p(시작번호 = 581544, 수집량 = 2973): # 100개당 1분정도 소요. Colab에서는 최대 1만개씩 수집 추천
    rtn = []
    wd = webdriver.Chrome('chromedriver',options=options)
    for idx, i in enumerate(tqdm(range(시작번호-수집량, 시작번호))):
        tmp = get_petition(wd, i)
        if idx == 0:
            rtn_c = list(tmp.keys())
        rtn.append(list(tmp.values()))
        # 오래 유지될경우 느려지는 경향이 있어 중간에 초기화
        if idx % 200 == 0:
            wd.quit()
            wd = webdriver.Chrome('chromedriver',options=options)
    return (rtn_c, rtn)


c, rtn = get_p()
%time wd = webdriver.Chrome('chromedriver',options=options)
%time rtn.append(list(get_petition(wd, 593681).values()))
%time rtn.append(list(get_petition(wd, 10).values()))
wd.quit()
결과 = pd.DataFrame(rtn, columns=c)
결과

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


578576 Message: 

578577 Message: 

578579 Message: 

578580 Message: 

578596 Message: 

578597 Message: 

578599 Message: 

578600 Message: 

578603 Message: 

578604 Message: 

578607 Message: 

578610 Message: 

578612 Message: 

578615 Message: 

578616 Message: 

578617 Message: 

578618 Message: 

578619 Message: 

578627 Message: 

578628 Message: 

578630 Message: 

578632 Message: 

578635 Message: 

578636 Message: 

578639 Message: 

578640 Message: 

578643 Message: 

578645 Message: 

578646 Message: 

578648 Message: 

578650 Message: 

578653 Message: 

578654 Message: 

578656 Message: 

578659 Message: 

578660 Message: 

578661 Message: 

578664 Message: 

578665 Message: 

578668 Message: 

578679 Message: 

578680 Message: 

578681 Message: 

578682 Message: 

578688 Message: 

578691 Message: 

578692 Message: 

578693 Message: 

578695 Message: 

578700 Message: 

578704 Message: 

578707 Message: 

578708 Message: 

578710 Message: 

578711 Message: 

578713 Mes

,number,progress,title,like,contents,answer,category,sday,eday,proponent
0,578571,청원종료,북핵협상에 관한 제안,3,하노이 회담이 실패했다. 실패한 가장 근본원인은 서로가 서로를 못믿는 불신이 가장 ...,,기타,2019-03-30,2019-04-29,naver
1,578572,청원종료,전북 과학 교육원 활성화를 위하여,1,전북과학교원원이 새롭게 잘 지어졌습니다.좋은 시설을 보다 많은 사람들이 활용했으면 ...,,육아/교육,2019-03-30,2019-04-29,naver
2,578573,청원종료,유튜브 서비스 중지 국민들의 밥줄 없앤다,18,유튜브를 없애는것은 현제 유튜브에서 영상을 올리는\n크리애이터들의 실직과 밥줄을 없...,,기타,2019-03-30,2019-04-29,naver
3,578574,청원종료,생활쓰레기 무단 소각을 적극적으로 계도및 처벌 부탁드립니다,9,"대도시는 불법으로 생활 쓰레기를 소각하는일이 있을수 없지만, 농어촌 곳곳에서 매일 ...",,농산어촌,2019-03-30,2019-04-29,naver
4,578575,청원종료,김의겸 비리는 제대로 방송도안하네.쓰레기집단들,31,현정부와 언론꼬라지를 눈뜨고 볼수가없네,,정치개혁,2019-03-30,2019-04-29,naver
...,...,...,...,...,...,...,...,...,...,...
2970,581541,청원종료,국토부 고위공직자 재산 형성과정을 조사해 주세요..,4941,오늘 신문기사를 보고 깜짝 놀라서 기절할 뻔 했습니다.\n경실련에서 고위 공무원들의...,,경제민주화,2019-07-18,2019-08-17,naver
2971,581542,청원종료,독립운동가들과 위안부 피해자들을 비난하는 악성 네티즌의 처벌을 요구합니다.,1596,인터넷 커뮤니티 '디시인사이드'에서 '*******' 라는 닉네임을 사용하는 조선족...,,기타,2019-07-18,2019-08-17,naver
2972,581543,청원종료,불공정거래 그림의 떡,206,-. (주) **중공업은 영세 하도급 업체로 2018년 1월부터 ******로부터 ...,,기타,2019-07-18,2019-08-17,naver
2973,593681,청원진행중,택배기사 주 5일(토요휴무) 근무를 도와주세요,6210,존경하는 국민 여러분 !\n\n저는 강산이 두번씩 변할만큼 오랜시간을 택배업계에 근...,,교통/건축/국토,2020-10-29,2020-11-28,naver


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
결과.to_csv('국민청원데이터.csv', header='true', encoding='utf-8')

[https://www1.president.go.kr/petitions/10](https://www1.president.go.kr/petitions/10)
- 관리자에 의해 비공개된 청원입니다